In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import random
from math import exp,log

In [2]:
def generateNormalVariables(n, normalVar):
    p = normalVar# number of normal predictors
    X = {}

    for j in range(p):
        np.random.seed(seed = j)   #seed for distribution
        random.seed(j)   #seed for random number generator
        
        # The following line generates random (not so random though :) mean and variance
        # for each predictor. We have to play with this and definitely this part needs work
        mu, sigma = j*(-random.random())**j,random.random()*(j + 0.5*(-1)**j) 
        
        X[j] = np.random.normal(mu, sigma, n)
    return X

In [3]:
def generateCountVariables(n, countVar, maxCount):
    X = {}
    
    for i in range(countVar):
        ### In this part we are generating variable, which takes discrete/count values
        ### Earlier we provided arbitary max count (for example: max number of visit to doctor is 6)
        ### Then I generate a population of (n x maxCount) entries. Each entry occurs n times in the pop
        ### Finally after shuffling the population, we take a random sample of n elements, which is X_i
        
        maxCt = maxCount[i]
        counts = np.arange(maxCt)
        population = np.repeat(counts , n)
        random.shuffle(population)
        X[i] = random.sample(list(population), n)
    return X

In [4]:
def generateCategoricalVariables(n, catVar, catNumber):
    temp = {}
    
    for i in range(catVar):
        ### We follow the same principle as count variables. 
        ### But, finally we convert the categorical variables to dummy variables
        ### Note: We use **** k dummy variables **** for k categories
        
        
        noOfCat = catNumber[i]
        categories = np.arange(noOfCat)
        population = np.repeat(categories , n)
        random.shuffle(population)
        temp[i] = random.sample(list(population), n)
    X = pd.DataFrame.from_dict(temp)
    X = pd.get_dummies(X,columns=X.columns,drop_first=False)
    return X

In [5]:
def generateRandomIntercept(n):
    ### For each observation (X) we generate an intercept
    np.random.seed(seed = 123)
    intercept =  np.random.normal(0, 1, n)*5
    return intercept

In [6]:

### The following function also needs some cleaning up. 
### It is just generating all types of variables and putting them together
### We also define how many observations, how many of each variable we need to generate here

def generateX():
    
    n = int(input('Enter number of observations:'))
#     normalVar = int(input('Enter number of variables normally distributed:'))
    normalVar = 5
    Xnorm = generateNormalVariables(n, normalVar)
    Xnorm = pd.DataFrame.from_dict(Xnorm)
    
    
#     catVar = int(input('Enter number of categorical variables:'))
#     catNumber = input('Enter the numbers separated by space: each number correspond to number of categories:')
#     catNumber = list(map(int, catNumber.split()))
    catVar = 1
    catNumber = [2]
    Xcat = generateCategoricalVariables(n, catVar, catNumber)#Make sure catvar = len(catNumber)
    
    
#     countVar = int(input('Enter number of count variables:'))
#     maxCount = input('Enter the numbers separated by space: each number correspond to maxCount:')
#     maxCount = list(map(int, maxCount.split()))
    countVar = 1
    maxCount = [4]
    Xcnt = generateCountVariables(n, countVar, maxCount)#Make sure countVar = len(maxCount)
    Xcnt = pd.DataFrame.from_dict(Xcnt)

    
    X = pd.concat([Xnorm,Xcat,Xcnt],ignore_index=True,axis=1)
    
    intercept = np.ones(n)#generateRandomIntercept(n)
    intercept = pd.DataFrame(intercept.reshape(n,1))
    X = pd.concat([intercept,X],axis = 1,ignore_index=True)#X with intercept
    return X

In [7]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [8]:
### As input with X matrix n x (p+1), and beta vector (p+1) x 1
### We also provide the name of the distribution as a quoted string

def generateResponseVariable(X, beta, dist):
    if dist == 'bernoulli':
        meanValues = sigmoid(X.dot(beta)) 
    elif dist == 'poisson':
        meanValues = np.exp(X.dot(beta))
    elif dist == 'exponential':
        meanValues = -1/(X.dot(beta))
    else:
        print('please spell check distribution name, all lowercase: bernoulli, poisson or exponential')
        
    
    meanValues = np.array([item for sublist in meanValues.values for item in sublist])
    y = []
    
    for eachMean in meanValues:
        np.random.seed(int(round(eachMean)))
        if dist == 'bernoulli':
            randomPrediction = np.random.binomial(1,eachMean)
        elif dist == 'poisson':
            randomPrediction = np.random.poisson(eachMean)
        elif dist == 'exponential':
            randomPrediction = np.random.exponential(eachMean)
        y.append(randomPrediction)
        print(randomPrediction, eachMean)
    
    return y

In [9]:
### This generates p random beta and 1 as intercept (which we can consider as true beta)
### This function needs work
### Either we have to find beta's, which work for all three distributions
### Or we can generate different sets of beta's for different distribution
### I think, same beta's would be better for comparison


def generateRandomBeta(p):
    beta = {}
    for j in range(p-1):
        random.seed(j)#seed for random number generator
        beta[j] = (j*0.5+1)*(-1)**j#random beta for each predictor
    
    beta = pd.DataFrame(list(beta.items()))
    beta = beta.drop([0],axis=1)
    
    beta.loc[-1] = 1
    beta.index = beta.index + 1  
    beta = beta.sort_index()# Beta with intercept
    return beta

In [10]:
def generateData(dist):
    X = generateX()

    n,p = X.shape
    beta = generateRandomBeta(p)

    y = generateResponseVariable(X, beta, dist) ### dist means pass distribution name as string
    return X, beta, y

In [178]:
[X, Beta, Y] = generateData('bernoulli')

Enter number of observations:5
0 1.1692643545821063e-05
1 0.7273516952021967
0 1.61862890530363e-07
1 0.999999999317384
0 2.092713501987904e-09


In [196]:
Y

[0, 1, 0, 1, 0]

In [139]:
def LikelihoodBinomial(x, beta, y):
    L = 0
    for i in range(len(y)):
        mu = x.loc[i].values.T.dot(beta)
        L += y[i]*mu - log(1+exp(mu))
    return L

In [138]:
def diffLikelihoodBinomial(x, beta, y):
    dL = 0
    for i in range(len(y)):
        mu = x.loc[i].values.T.dot(beta)
        dL += (y[i] - (1/(1+exp(-mu))))* x.loc[i]
    return dL

In [144]:
def diff2Binomial(x,beta,y):
    d2L = 0
    #S = pd.DataFrame(np.zeros((len(y), len(y))))
    
    for i in range(len(y)):
        mu = x.loc[i].values.T.dot(beta)
        #S[i][i] = mu(1-mu)
        S = mu*(1-mu)
        d2L += (x.loc[i].values.T.dot(x.loc[i].values))*S
    return d2L
    

In [140]:
LikelihoodBinomial(X,Beta,Y)

array([-0.31835701])

In [141]:
diffLikelihoodBinomial(X,Beta,Y)

0    2.726364e-01
1    4.133936e-02
2   -1.073142e-01
3    4.620410e-01
4    1.508768e-01
5    6.666742e-02
6    2.726366e-01
7   -1.611803e-07
8   -1.218383e-05
Name: 0, dtype: float64

In [146]:
1/diff2Binomial(X,Beta,Y)

array([-3.03264976e-05])

In [307]:
def UpdateBeta(x,beta,y):
    beta = beta[1]
    print(beta)
    InverseDiff2 = diff2Binomial(x,beta,y)
    for i in range(len(y)):
        mu = x.loc[i].values.T.dot(beta)
        xSx = (x.loc[i].values.T.dot(x.loc[i].values))*S
        z = (1/xSx)*(y[i] - mu)
        z = z[1]
        beta += (x.loc[i].values)*z
    return beta

In [309]:
UpdateBeta(X,Beta,Y)

0   -1.101642e+07
1   -7.852132e+06
2   -2.150992e+06
3    2.284154e+07
4    5.606072e+06
5    1.765265e+06
6   -1.051281e+07
7   -5.036126e+05
8   -3.209461e+07
Name: 1, dtype: float64


0   -2.722134e+13
1   -1.940242e+13
2   -5.314955e+12
3    5.644026e+13
4    1.385257e+13
5    4.361880e+12
6   -2.597682e+13
7   -1.244522e+12
8   -7.930482e+13
Name: 1, dtype: float64

In [175]:
InverseDiff2 = diff2Binomial(X,Beta,Y)

In [200]:
Beta.values

array([[ 1. ],
       [ 1. ],
       [-1.5],
       [ 2. ],
       [-2.5],
       [ 3. ],
       [-3.5],
       [ 4. ],
       [-4.5]])

In [168]:
print(InverseDiff2)
mu = X.loc[1].values.T.dot(Beta)
print(mu)

[-32974.46388573]
[0.98122742]


In [169]:
(X.loc[1].values.T.dot(X.loc[1].values))*S

1    0.099763
dtype: float64

In [195]:
Beta[1] +(X.loc[1].values.T *(Y[1] - mu))

0    1.018773
1    1.002847
2   -1.507388
3    2.031813
4   -2.489610
5    3.004590
6   -3.481227
7    4.000000
8   -4.500000
Name: 1, dtype: float64

[[ 1. ]
 [ 1. ]
 [-1.5]
 [ 2. ]
 [-2.5]
 [ 3. ]
 [-3.5]
 [ 4. ]
 [-4.5]]
[-11.35653898]
1    0.187674
dtype: float64
1    5.328388
dtype: float64


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'